In [19]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time

In [21]:
def scrape_category(category_url, category_title, csvwriter, visited_urls):
    if category_url in visited_urls:
        return
    visited_urls.add(category_url)
    
    logging.info(f"Visiting category URL: {category_url}")
    
    try:
        driver.get(category_url)
        time.sleep(5)  # Increase wait time to ensure the page fully loads
        
        # Parse the page with BeautifulSoup
        page_soup = BeautifulSoup(driver.page_source, 'html.parser')
        content = page_soup.find('div', id='wrapper', class_='wrapper')
        
        if not content:
            logging.error(f"No content found on {category_url}")
            return
        
        # Extract all paragraphs
        paragraphs = ' '.join([p.get_text(strip=True) for p in content.find_all('p')])
        
        # Write the category content to the CSV
        csvwriter.writerow([category_url, category_title, paragraphs])
        logging.info(f"Successfully scraped category URL: {category_url}")
        
        # Find and follow sub-links within the category
        sub_links = content.find_all('a', href=True)
        for sub_link in sub_links:
            sub_href = sub_link['href']
            if sub_href.startswith('/'):
                sub_full_url = "https://www.gov.uk" + sub_href
                scrape_section(sub_full_url, category_title, csvwriter, visited_urls)
            elif sub_href.startswith('http'):
                scrape_section(sub_href, category_title, csvwriter, visited_urls)
    
    except Exception as e:
        logging.error(f"An error occurred while scraping category {category_url}: {e}")

def scrape_section(section_url, category_title, csvwriter, visited_urls):
    if section_url in visited_urls:
        return
    visited_urls.add(section_url)
    
    logging.info(f"Visiting section URL: {section_url}")
    
    try:
        driver.get(section_url)
        time.sleep(5)  # Increase wait time to ensure the page fully loads
        
        # Parse the page with BeautifulSoup
        page_soup = BeautifulSoup(driver.page_source, 'html.parser')
        content = page_soup.find('div', id='wrapper', class_='wrapper')
        
        if not content:
            logging.error(f"No content found on {section_url}")
            return
        
        # Extract the title from the <h1> tag
        title_tag = content.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else 'No Title'
        
        # Extract all paragraphs
        paragraphs = ' '.join([p.get_text(strip=True) for p in content.find_all('p')])
        
        # Write the section content to the CSV
        csvwriter.writerow([section_url, title, paragraphs])
        logging.info(f"Successfully scraped section URL: {section_url}")
        
        # Find and follow sub-links within the section
        sub_links = content.find_all('a', href=True)
        for sub_link in sub_links:
            sub_href = sub_link['href']
            if sub_href.startswith('/'):
                sub_full_url = "https://www.gov.uk" + sub_href
                scrape_section(sub_full_url, category_title, csvwriter, visited_urls)
            elif sub_href.startswith('http'):
                scrape_section(sub_href, category_title, csvwriter, visited_urls)
    
    except Exception as e:
        logging.error(f"An error occurred while scraping section {section_url}: {e}")

def main(url, output_csv, log_file):
    # Set up logging
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Set up Selenium WebDriver for Firefox
    options = Options()
    options.headless = True
    service = Service('/opt/homebrew/bin/geckodriver')  # Use the correct path
    global driver
    driver = webdriver.Firefox(service=service, options=options)
    
    try:
        # Open the output CSV file
        with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
            csvwriter = csv.writer(outfile)
            csvwriter.writerow(['Section', 'Title', 'Content'])
            
            # Start scraping from the main URL
            visited_urls = set()
            scrape_category(url, 'Main Page', csvwriter, visited_urls)
    
    except Exception as e:
        logging.error(f"An error occurred in the main function: {e}")
    
    finally:
        driver.quit()

# Example usage
url = "https://www.gov.uk/browse/visas-immigration"
output_csv = "gov_additional_content.csv"
log_file = "scrape_errors.log"
main(url, output_csv, log_file)